In [99]:
from cv2 import VideoCapture, imwrite, resize, CAP_PROP_POS_MSEC, CAP_PROP_FRAME_COUNT, CAP_PROP_FRAME_HEIGHT, CAP_PROP_FRAME_WIDTH, CAP_PROP_FPS, VideoWriter, VideoWriter_fourcc, imread, cvtColor
import os
from os import path
import time
from pathlib import Path
import queue
import numpy as np

!pip install pytube
!pip install moviepy
from moviepy.editor import VideoFileClip
from pytube import YouTube, Stream

In [100]:
"""
Retrieves video from youtube in smallest mp4 format available and saves it to videos folder in working directory
"""
def get_video_youtube(url):

    current_path = Path.cwd()

    """
    ensures video folder exists in project root directory
    """
    save_path = current_path / "videos"

    if not Path.exists(save_path):
        Path.mkdir(save_path)

    """
    Gets youtube object, identifies the smallest stream and downloads it
    """
    youtube_object = YouTube(url)
    video_path = youtube_object.streams.filter(progressive=True, file_extension='mp4').get_lowest_resolution().download(save_path.__str__())

    return video_path

In [101]:
path = Path(get_video_youtube("https://www.youtube.com/watch?v=PSf09R3D7Lo&ab_channel=Panasonicsecurity"))

In [102]:
"""
Checks that the path goes to a file
"""
def check_file_path(video_path):
    if video_path == None:
        print("No video file provided")
        return False    

    if not Path.exists(video_path):
        print("Invalid Path")
        return False

    if not Path.is_file(video_path):
        print("Input must be file")
        return False

    return True

In [103]:
"""
Get and reutrns the details for the video pointed to by the file path
"""
def video_details(video_path):
    check_file_path(video_path)

    video = VideoCapture(video_path)

    if not video.isOpened():
        print("Unable to open file. Please ensure input video is either .AVI or .MKV")
        return False

    total_video_frames = video.get(CAP_PROP_FRAME_COUNT)
    frame_height = video.get(CAP_PROP_FRAME_HEIGHT)
    frame_width = video.get(CAP_PROP_FRAME_WIDTH)
    video_fps = video.get(CAP_PROP_FPS)
    video_length = total_video_frames / video_fps
    print("Total Frames in Video:",total_video_frames)
    print("Frame Height:",frame_height)
    print("Frame Width:",frame_width)
    print("Frames per Second:",video_fps)  
    print("Video Length",video_length)

    video.release()

    return total_video_frames, video_fps, frame_height, frame_width, video_length

In [104]:
def image_extraction_to_queue(video_path, frame_queue, image_per_second=10, target_height=256, target_width=512, frame_count=0, save_images = "off"):
    """
    Checks validity of provided video file path
    """
    # if(check_file_path(video_path) == False):
    #     return False

    """
    loads video
    """
    # video = VideoCapture(video_path)

    # if not video.isOpened():
    #     print("Unable to open file. Please ensure input video is either .AVI or .MKV")
    #     return False

    start = time.time()

    clip = VideoFileClip(video_path)
    video_length = clip.duration
    print("Video Length According to MOVIEPY: ", clip.duration)

    """
    iterates through loaded video and extracts frames
    at intervals dictated by save_interal 
    """
    if save_images == "on":
        video_path = Path(video_path)

        video_name = video_path.stem

        """
        gets working directory
        """
        current_path = Path.cwd()

        """
        ensures image folder exists in project root directory
        """

        image_path = current_path / "images"

        if not Path.is_dir(image_path):
            Path.mkdir(image_path)

        image_path = image_path / video_name

        if not Path.is_dir(image_path):
            Path.mkdir(image_path)

    actual_frames = 0

    scale = (target_width, target_height)
    # image_transforms = transforms.Compose(
    #     [transforms.Resize((target_height, target_width)), transforms.ToTensor()])
    last_frame_time = -10000000

    time_inc = 1 / image_per_second

    for vid_time in np.arange(0, video_length, time_inc):

        frame = clip.get_frame(vid_time)
        frame = resize(frame, scale)

        if frame.ndim != 3:
            frame = np.expand_dims(frame, axis = 2)
            frame = np.repeat(frame, 3, axis = 2)
            print("Grayscale changed back to", frame.shape)
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # t_image = image_transforms(Image.fromarray(img))
        # frame_queue.put((frame_count, frame, t_image))
        if save_images == "on":
            image_name = image_path / (video_name.__str__() + "_frame_{:08d}.jpg".format(frame_count))
            imwrite(image_name.__str__(), frame)
        frame_count += 1

    # video.release()
    end = time.time()

    print("Extraction took: ", end - start, "Seconds     Frame Count:", frame_count, "Total Frames in Video", actual_frames)

    return frame_count

In [105]:
    # video_path = r"C:\Users\Bob\Videos\Project_1.avi"

    size = (256,512)
    fps = 10

    frame_queue = queue.Queue()

    # video_path = get_video_youtube("https://www.youtube.com/watch?v=MNn9qKG2UFI&ab_channel=KarolMajek")
    # print(video_path)
    # image_path = image_extraction("int-camera-125--7.mp4", fps, size[0], size[1])
    image_extraction_to_queue("hwy-camera-944--9.mp4", frame_queue, image_per_second=10, target_height=256, target_width=512, frame_count=0, save_images = "on")
    

Video Length According to MOVIEPY:  304.33
Extraction took:  20.749000072479248 Seconds     Frame Count: 3044 Total Frames in Video 0


3044

In [108]:
def image_extraction_to_queue2(video_path, frame_queue, image_per_second=10, target_height=256, target_width=512, frame_count=0, save_images = "off"):
    
    start = time.time()

    """
    Checks validity of provided video file path
    """
    # if(check_file_path(video_path) == False):
    #     return False

    """
    loads video
    """
    video = VideoCapture(video_path)

    if not video.isOpened():
        print("Unable to open file. Please ensure input video is either .AVI or .MKV")
        return False

    clip = VideoFileClip(video_path)
    print("Video Length According to MOVIEPY: ", clip.duration)

    """
    iterates through loaded video and extracts frames
    at intervals dictated by save_interal 
    """
    if save_images == "on":
        video_path = Path(video_path)

        video_name = video_path.stem

        """
        gets working directory
        """
        current_path = Path.cwd()

        """
        ensures image folder exists in project root directory
        """

        image_path = current_path / "images"

        if not Path.is_dir(image_path):
            Path.mkdir(image_path)

        image_path = image_path / video_name

        if not Path.is_dir(image_path):
            Path.mkdir(image_path)

    actual_frames = 0

    scale = (target_width, target_height)
    # image_transforms = transforms.Compose(
    #     [transforms.Resize((target_height, target_width)), transforms.ToTensor()])
    last_frame_time = -10000000
    while True:

        valid, frame = video.read()

        if not valid:
            # print("END at Frame Count", frame_count, " With save Interval of", save_interval)
            break

        if(video.get(CAP_PROP_POS_MSEC) - last_frame_time >= 1000 / image_per_second):
            last_frame_time = video.get(CAP_PROP_POS_MSEC)
            frame = resize(frame, scale)
            # frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            
            # print("Image Changed to", frame.shape)
            if frame.ndim != 3:
                frame = np.expand_dims(frame, axis = 2)
                frame = np.repeat(frame, 3, axis = 2)
                print("Grayscale changed back to", frame.shape)
            # img = cvtColor(frame, cv2.COLOR_BGR2RGB)
            # t_image = image_transforms(Image.fromarray(img))
            # frame_queue.put((frame_count, frame, t_image))
            if save_images == "on":
                image_name = image_path / (video_name.__str__() + "_frame_{:08d}.jpg".format(frame_count))
                imwrite(image_name.__str__(), frame)
            frame_count += 1

        actual_frames += 1

    video.release()
    end = time.time()

    print("Extraction took: ", end - start, "Seconds     Frame Count:", frame_count, "Total Frames in Video", actual_frames)

    return frame_count

In [109]:
image_extraction_to_queue2("hwy-camera-944--9.mp4", frame_queue, image_per_second=10, target_height=256, target_width=512, frame_count=0, save_images = "on")

Video Length According to MOVIEPY:  304.33
Extraction took:  16.341997385025024 Seconds     Frame Count: 2757 Total Frames in Video 9123


2757

In [59]:
video_name = image_path.split("/")

NameError: name 'image_path' is not defined

In [ ]:
def make_video(image_path, size, fps):

    current_path = os.getcwd()

    images = []
    names = []

    for image in os.listdir(image_path):
        names.append(image)
        img = imread(os.path.join(image_path, image))
        images.append(img)

    out = [x for _, x in sorted(zip(names,images))]

    if not path.isdir(current_path + r"\generatedvideos"):
        os.mkdir(current_path + r"\generatedvideos")

    name = image_path.split("\\")
    print(name[len(name)-1])

    video_path = current_path + r"\generatedvideos\\" + name[len(name)-1] + r".avi"

    codex = VideoWriter_fourcc(*'MP4V')
    writer = VideoWriter(video_path, codex, fps, (size[1],size[0]))

    for image in out:
        # cv2_imshow(image)
        writer.write(image)
    writer.release()

In [ ]:
make_video(image_path, size, fps)